In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from copy import deepcopy
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sys 
import time

BATCH_SIZE = 10

print(tf.__version__)

2.7.0


In [3]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/1-3-5-Sem-Derivados.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/2-4-6-Sem-Derivados.csv"
df = pd.read_csv(path_training)
df_test = pd.read_csv(path_test)
df


,ServerPort,15,22,45,59,60,82,83,84,target
0,80,373,359,1,1,1,140,319,140,WWW
1,80,373,359,1,1,1,141,319,141,WWW
2,80,376,362,1,1,1,141,322,141,WWW
3,80,373,359,1,1,1,142,319,142,WWW
4,80,379,365,1,1,1,141,325,141,WWW
...,...,...,...,...,...,...,...,...,...,...
69438,80,1514,1500,1,1,4,1448,354,366,WWW
69439,80,1514,1500,1,1,1,1448,350,1009,WWW
69440,80,1434,1420,1,1,1,1380,433,784,WWW
69441,80,1434,1420,1,1,1,1380,269,295,WWW


In [4]:
#df.dtypes

In [5]:
server_port = df.pop('ServerPort')
target = df.pop('target')



server_port_test = df_test.pop('ServerPort')
target_test = df_test.pop('target')


df

,15,22,45,59,60,82,83,84
0,373,359,1,1,1,140,319,140
1,373,359,1,1,1,141,319,141
2,376,362,1,1,1,141,322,141
3,373,359,1,1,1,142,319,142
4,379,365,1,1,1,141,325,141
...,...,...,...,...,...,...,...,...
69438,1514,1500,1,1,4,1448,354,366
69439,1514,1500,1,1,1,1448,350,1009
69440,1434,1420,1,1,1,1380,433,784
69441,1434,1420,1,1,1,1380,269,295


In [6]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes

In [7]:
#Converts the above list of classes to an array of index
target.unique()
#target = target.apply(lambda x: classes.index(x))
#target.unique()
#target_test = target_test.apply(lambda x: classes.index(x))

array(['WWW', 'MAIL', 'FTP-CONTROL', 'FTP-PASV', 'ATTACK', 'P2P',
       'DATABASE', 'FTP-DATA', 'MULTIMEDIA', 'SERVICES', 'INTERACTIVE'],
      dtype=object)

In [8]:
numeric_feature_names = ['15','22','45','59','60','82','83','84']
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing
numeric_features.head()

,15,22,45,59,60,82,83,84
0,373,359,1,1,1,140,319,140
1,373,359,1,1,1,141,319,141
2,376,362,1,1,1,141,322,141
3,373,359,1,1,1,142,319,142
4,379,365,1,1,1,141,325,141


In [9]:
#converting pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features_test = tf.convert_to_tensor(numeric_features_test)
numeric_features

<tf.Tensor: shape=(69443, 8), dtype=int64, numpy=
array([[ 373,  359,    1, ...,  140,  319,  140],
       [ 373,  359,    1, ...,  141,  319,  141],
       [ 376,  362,    1, ...,  141,  322,  141],
       ...,
       [1434, 1420,    1, ..., 1380,  433,  784],
       [1434, 1420,    1, ..., 1380,  269,  295],
       [1514, 1500,    7, ...,  144,    6,   35]])>

In [10]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [11]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='tanh'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

Requisição Weverton 
Como recuperar os pesos após os treinos? Descobrir o método

In [12]:
model = get_basic_model()
model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE)

Epoch 1/10


UnimplementedError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 

print('\nTest accuracy:', test_acc)


In [ ]:
print('\nPesos da primeira camada com inputs normalizados')
print(model.layers[0].weights)
print('\nPesos da segunda camada: intermediária')
print(model.layers[1].weights)
print('\nPesos da terceira camada: final')
print(model.layers[2].weights)